In [1]:
import tensorflow as tf
import efficientnet.tfkeras as efn
import re
import numpy as np
import random
import ast
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                          BatchNormalization, Input, Conv2D, GlobalAveragePooling2D,concatenate,Concatenate,multiply, LocallyConnected2D, Lambda)
# from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from tf_explain.callbacks.grad_cam import GradCAMCallback
import tensorflow_addons as tfa
from TPUDataLoaderRNN import DataLoader
import cv2
tf.random.set_seed(2020)

In [2]:
# Create strategy from tpu
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='smas255')
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Initializing the TPU system: smas255


INFO:tensorflow:Initializing the TPU system: smas255


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
FLAGS ={}
# Configuration
FLAGS['FOLD']='0'
FLAGS['W'] = 224
FLAGS['H'] = 224
FLAGS['EPOCHS'] = 40
FLAGS['LR'] = 0.00005
FLAGS['WEIGHT_DECAY']=1e-4
FLAGS['BATCH_SIZE'] =  32 * strategy.num_replicas_in_sync
FLAGS['VAL_BATCH_SIZE'] = 16
FLAGS['GCS_PATH'] = 'gs://deepfake_images/TFRecords/rnn/'
FLAGS['GCS_LOGS'] = 'gs://deepfake_images/trainingLogs/'
FLAGS['TRAINING_FILENAMES'] = tf.io.gfile.glob(FLAGS['GCS_PATH'] + 'fold{}/train/*.tfrec'.format(FLAGS['FOLD']))
FLAGS['VALIDATION_FILENAMES'] = tf.io.gfile.glob(FLAGS['GCS_PATH'] + 'fold{}/val/*.tfrec'.format(FLAGS['FOLD']))
FLAGS['TEST_FILENAMES'] = tf.io.gfile.glob(FLAGS['GCS_PATH'] + 'test/*.tfrec')

random.shuffle(FLAGS['TRAINING_FILENAMES'])


In [4]:
dataLoader = DataLoader(FLAGS)

In [5]:
# img_list = []
# lbl_list = []
# for img,lbl in dataLoader.get_gradcam_dataset():
#     img_list = img.numpy()
#     lbl_list = lbl.numpy()
#     break
# np.save('./data/img.npy',img_list)
# np.save('./data/lbl.npy',lbl_list)

In [6]:
# img_list = np.load('./data/img.npy')
# lbl_list = np.load('./data/lbl.npy')

In [7]:
from tensorflow.keras.layers import *
def efficientModel(weights_path=None):
    enet = efn.EfficientNetB0(
        weights='noisy-student',
        include_top=False,
        pooling = 'avg'
    )
#     if weights_path!=None:
#         model.load_weights('./b0.h5')
    #output = tf.keras.layers.GlobalAveragePooling2D()(enet.output)
    #output = tf.keras.layers.Dropout(0.2)(enet.output)
    
    inp=Input((16,FLAGS['H'],FLAGS['W'],3))
    x=TimeDistributed(enet)(inp)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = LSTM(64)(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = Dense(16, activation='elu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    output = Dense(1,activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inp, outputs=output)
    
    return model

In [8]:
with strategy.scope():
    model = efficientModel()
#     model.load_weights('./rnn_b0.h5')

In [9]:
FLAGS['NUM_TRAINING_IMAGES'] = dataLoader.count_data_items(FLAGS['TRAINING_FILENAMES'])
FLAGS['NUM_VALIDATION_IMAGES'] = dataLoader.count_data_items(FLAGS['VALIDATION_FILENAMES'])
FLAGS['STEPS_PER_EPOCH'] = FLAGS['NUM_TRAINING_IMAGES'] // FLAGS['BATCH_SIZE']
FLAGS['VALIDATION_STEPS'] = FLAGS['NUM_VALIDATION_IMAGES'] // FLAGS['VAL_BATCH_SIZE']
print('Dataset: {} training images, {} validation images, {} Steps'.format(
    FLAGS['NUM_TRAINING_IMAGES'], FLAGS['NUM_VALIDATION_IMAGES'],FLAGS['STEPS_PER_EPOCH']))

Dataset: 66008 training images, 16677 validation images, 257 Steps


In [10]:
class_weight={0:2.43284682,1:0.62934289}

## Warm up

In [11]:
def warmup():
    # warm up model
    for layer in model.layers:
        layer.trainable = False

    for i in range(-3,0):
        model.layers[i].trainable = True
    model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            #loss=macro_double_soft_f1,
            #optimizer=tf.keras.optimizers.SGD(learning_rate= FLAGS['LR'],momentum=0.9,decay=FLAGS['WEIGHT_DECAY']),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
    model.fit(
        dataLoader.get_training_dataset(), 
        steps_per_epoch=FLAGS['STEPS_PER_EPOCH'],
        class_weight=class_weight,
        epochs=1)
    #model.save_weights('./rnn_b0.h5')

In [12]:
WARM_UP = True
if WARM_UP:
    warmup()

Train for 257 steps
257/257 [==============================] - 481s 2s/step - loss: 0.6939 - accuracy: 0.5464


In [13]:
def build_lrfn(lr_start=0.00001, lr_max=0.000075, 
               lr_min=0.000001, lr_rampup_epochs=5, 
               lr_sustain_epochs=3, lr_exp_decay=.8):
    lr_max = lr_max * strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
    
    return lrfn

## Train Model

In [ ]:
for layer in model.layers:
    layer.trainable = True
radam = tfa.optimizers.RectifiedAdam(lr=FLAGS['LR'])
# opt = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)
lrfn = build_lrfn()
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1) 
model.compile(
        optimizer=radam,#tf.keras.optimizers.Adam(lr=FLAGS['LR'], clipnorm=1.),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
#lr_schedule = OneCycleScheduler(FLAGS['LR'], FLAGS['STEPS_PER_EPOCH'])
# lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=2, 
#                                    verbose=1, mode='auto', min_delta=0.0001)
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=FLAGS['GCS_LOGS'], update_freq=10)
# gradcam_callback = GradCAMCallback(
#         validation_data=(img_list,lbl_list),
#         layer_name="top_conv",
#         class_index=0,
#         output_dir=FLAGS['GCS_LOGS']
#     )
#lr_schedule = OneCycleScheduler(FLAGS['LR'], FLAGS['STEPS_PER_EPOCH'])
#lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='gs://hamid_kaggles/model_checkpoints/',
                                                 monitor='val_loss',
                                                 verbose=1,
                                                 save_best_only=True,
                                                 save_weights_only=True
                                                 )
history = model.fit(
    dataLoader.get_training_dataset(), 
    steps_per_epoch=FLAGS['STEPS_PER_EPOCH'],
    epochs=FLAGS['EPOCHS'], 
    callbacks=[lr_schedule,cp_callback],
    class_weight=class_weight,
    validation_data=dataLoader.get_validation_dataset(),
    validation_freq=2)

Train for 257 steps

Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 1/40
256/257 [============================>.] - ETA: 1s - loss: 0.6940 - accuracy: 0.4645WARNING:tensorflow:Can save best model only with val_loss available, skipping.


257/257 [==============================] - 440s 2s/step - loss: 0.6940 - accuracy: 0.4646

Epoch 00002: LearningRateScheduler reducing learning rate to 0.000128.
Epoch 2/40
256/257 [============================>.] - ETA: 1s - loss: 0.5488 - accuracy: 0.6963
Epoch 00002: val_loss improved from inf to 0.48667, saving model to gs://hamid_kaggles/model_checkpoints/
257/257 [==============================] - 488s 2s/step - loss: 0.5483 - accuracy: 0.6964 - val_loss: 0.4867 - val_accuracy: 0.8317

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00024599999999999996.
Epoch 3/40
256/257 [============================>.] - ETA: 1s - loss: 0.3212 - accuracy: 0.8497WARNING:tensorflow:Can save best model only with val_loss available, skipping.


257/257 [==============================] - 344s 1s/step - loss: 0.3209 - accuracy: 0.8500

Epoch 00004: LearningRateScheduler reducing learning rate to 0.00036399999999999996.
Epoch 4/40
256/257 [============================>.] - ETA: 1s - loss: 0.2185 - accuracy: 0.8970
Epoch 00004: val_loss did not improve from 0.48667
257/257 [==============================] - 371s 1s/step - loss: 0.2184 - accuracy: 0.8971 - val_loss: 0.4926 - val_accuracy: 0.8712

Epoch 00005: LearningRateScheduler reducing learning rate to 0.00048199999999999995.
Epoch 5/40
256/257 [============================>.] - ETA: 1s - loss: 0.1698 - accuracy: 0.9296WARNING:tensorflow:Can save best model only with val_loss available, skipping.


257/257 [==============================] - 349s 1s/step - loss: 0.1695 - accuracy: 0.9297

Epoch 00006: LearningRateScheduler reducing learning rate to 0.0006.
Epoch 6/40
256/257 [============================>.] - ETA: 1s - loss: 0.1465 - accuracy: 0.9371
Epoch 00006: val_loss improved from 0.48667 to 0.42162, saving model to gs://hamid_kaggles/model_checkpoints/
257/257 [==============================] - 371s 1s/step - loss: 0.1466 - accuracy: 0.9373 - val_loss: 0.4216 - val_accuracy: 0.8904

Epoch 00007: LearningRateScheduler reducing learning rate to 0.0006.
Epoch 7/40
256/257 [============================>.] - ETA: 1s - loss: 0.1141 - accuracy: 0.9489WARNING:tensorflow:Can save best model only with val_loss available, skipping.


257/257 [==============================] - 347s 1s/step - loss: 0.1141 - accuracy: 0.9486

Epoch 00008: LearningRateScheduler reducing learning rate to 0.0006.
Epoch 8/40
 90/257 [=========>....................] - ETA: 3:49 - loss: 0.1026 - accuracy: 0.9538

In [ ]:
import matplotlib.pyplot as plt
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [30]:
model.save_weights('./b3.h5')

## Test Model

In [27]:
with strategy.scope():
    model = efficientAttention()
    model.load_weights('gs://hamid_kaggles/model_checkpoints/')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=FLAGS['LR'], clipnorm=1.),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

In [28]:
FLAGS['NUM_TEST_IMAGES'] = dataLoader.count_data_items(FLAGS['TEST_FILENAMES'])

FLAGS['TEST_STEPS_PER_EPOCH'] = FLAGS['NUM_TEST_IMAGES'] // FLAGS['BATCH_SIZE']


In [29]:
results = model.evaluate(dataLoader.get_test_dataset(), 
    steps=FLAGS['TEST_STEPS_PER_EPOCH'],)

14/14 [==============================] - 16s 1s/step - loss: 0.2988 - accuracy: 0.8627


In [12]:
!wget https://tpu.googleapis.com/v1/projects/deepfake-267708/locations/europe-west4-a/nodes/tpu-32?alt=json

--2020-03-08 02:48:41--  https://tpu.googleapis.com/v1/projects/deepfake-267708/locations/europe-west4-a/nodes/tpu-32?alt=json
Resolving tpu.googleapis.com (tpu.googleapis.com)... 108.177.119.95, 2a00:1450:4013:c04::5f
Connecting to tpu.googleapis.com (tpu.googleapis.com)|108.177.119.95|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2020-03-08 02:48:41 ERROR 403: Forbidden.



In [14]:
!python deepfake_training.py

2020-03-17 00:55:59.249643: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-03-17 00:55:59.249754: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-03-17 00:55:59.249779: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2020-03-17 00:56:00.731180: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2020-03-17 00:56:00.731232: E tensorflow/stream_executor/cuda

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import logging

logging.getLogger('tensorflow').setLevel(logging.ERROR)

from tensorflow.keras.callbacks import Callback

class CosineAnnealer:
    
    def __init__(self, start, end, steps):
        self.start = start
        self.end = end
        self.steps = steps
        self.n = 0
        
    def step(self):
        self.n += 1
        cos = np.cos(np.pi * (self.n / self.steps)) + 1
        return self.end + (self.start - self.end) / 2. * cos


class OneCycleScheduler(Callback):


    def __init__(self, lr_max, steps, mom_min=0.85, mom_max=0.95, phase_1_pct=0.3, div_factor=25.):
        super(OneCycleScheduler, self).__init__()
        lr_min = lr_max / div_factor
        final_lr = lr_max / (div_factor * 1e4)
        phase_1_steps = steps * phase_1_pct
        phase_2_steps = steps - phase_1_steps
        
        self.phase_1_steps = phase_1_steps
        self.phase_2_steps = phase_2_steps
        self.phase = 0
        self.step = 0
        
        self.phases = [[CosineAnnealer(lr_min, lr_max, phase_1_steps), CosineAnnealer(mom_max, mom_min, phase_1_steps)], 
                 [CosineAnnealer(lr_max, final_lr, phase_2_steps), CosineAnnealer(mom_min, mom_max, phase_2_steps)]]
        
        self.lrs = []
        self.moms = []

    def on_train_begin(self, logs=None):
        self.phase = 0
        self.step = 0

        self.set_lr(self.lr_schedule().start)
        self.set_momentum(self.mom_schedule().start)
        
    def on_train_batch_begin(self, batch, logs=None):
        self.lrs.append(self.get_lr())
        self.moms.append(self.get_momentum())

    def on_train_batch_end(self, batch, logs=None):
        self.step += 1
        if self.step >= self.phase_1_steps:
            self.phase = 1
            
        self.set_lr(self.lr_schedule().step())
        self.set_momentum(self.mom_schedule().step())
        
    def get_lr(self):
        try:
            return tf.keras.backend.get_value(self.model.optimizer.lr)
        except AttributeError:
            return None
        
    def get_momentum(self):
        try:
            return tf.keras.backend.get_value(self.model.optimizer.momentum)
        except AttributeError:
            return None
        
    def set_lr(self, lr):
        try:
            tf.keras.backend.set_value(self.model.optimizer.lr, lr)
        except AttributeError:
            pass # ignore
        
    def set_momentum(self, mom):
        try:
            tf.keras.backend.set_value(self.model.optimizer.momentum, mom)
        except AttributeError:
            pass # ignore

    def lr_schedule(self):
        return self.phases[self.phase][0]
    
    def mom_schedule(self):
        return self.phases[self.phase][1]
    
    def plot(self):
        ax = plt.subplot(1, 2, 1)
        ax.plot(self.lrs)
        ax.set_title('Learning Rate')
        ax = plt.subplot(1, 2, 2)
        ax.plot(self.moms)
        ax.set_title('Momentum')